# Policy gradient methods on CartPole

In [1]:
import gym
import ptan
import numpy as np
from tensorboardX import SummaryWriter
from typing import Optional

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

Besides the already familiar hyperparameters, we have two new ones: the ENTROPY_
BETA value is the scale of the entropy bonus and the REWARD_STEPS value specifies
how many steps ahead the Bellman equation is unrolled to estimate the discounted
total reward of every transition.

In [2]:
GAMMA = 0.99
LEARNING_RATE = 0.001
ENTROPY_BETA = 0.01
BATCH_SIZE = 8

REWARD_STEPS = 10

The network architecture is exactly the same as in the previous examples for
CartPole: a two-layer network with 128 neurons in the hidden layer. The preparation
code is also the same as before, except the experience source is asked to unroll
the Bellman equation for 10 steps.
The following is the part that differs from 04_cartpole_pg.py:

``` python
exp_source = ptan.experience.ExperienceSourceFirstLast(
env, agent, gamma=GAMMA, steps_count=REWARD_STEPS)
```

In [3]:
class PGN(nn.Module):
    def __init__(self, input_size, n_actions):
        super(PGN, self).__init__()

        self.net = nn.Sequential(
            nn.Linear(input_size, 128),
            nn.ReLU(),
            nn.Linear(128, n_actions)
        )

    def forward(self, x):
        return self.net(x)

In [4]:
def smooth(old: Optional[float], val: float, alpha: float = 0.95) -> float:
    if old is None:
        return val
    return old * alpha + (1-alpha)*val

In the loss calculation, we use the same code as before to calculate the policy loss
(which is the negated policy gradient):

``` python
optimizer.zero_grad()
logits_v = net(states_v)
log_prob_v = F.log_softmax(logits_v, dim=1)
log_prob_actions_v = batch_scale_v * log_prob_v[range(BATCH_SIZE),batch_actions_t]
loss_policy_v = -log_prob_actions_v.mean()
```

Then we add the entropy bonus to the loss by calculating the entropy of the batch
and subtracting it from the loss. As entropy has a maximum for uniform probability
distribution and we want to push the training toward this maximum, we need to
subtract from the loss.

``` python
prob_v = F.softmax(logits_v, dim=1)
entropy_v = -(prob_v * log_prob_v).sum(dim=1).mean()
entropy_loss_v = -ENTROPY_BETA * entropy_v
loss_v = loss_policy_v + entropy_loss_v
loss_v.backward()
optimizer.step()
```

Then, we calculate the Kullback-Leibler (KL) divergence between the new policy
and the old policy. KL divergence is an information theory measurement of how one
probability distribution diverges from another expected probability distribution. In
our example, it is being used to compare the policy returned by the model before and
after the optimization step. High spikes in KL are usually a bad sign, showing that
our policy was pushed too far from the previous policy, which is a bad idea most of
the time (as our NN is a very nonlinear function in a high-dimensional space, such
large changes in the model weight could have a very strong influence on the policy).

``` python
new_logits_v = net(states_v)
new_prob_v = F.softmax(new_logits_v, dim=1)
kl_div_v = -((new_prob_v / prob_v).log() *
prob_v).sum(dim=1).mean()
writer.add_scalar("kl", kl_div_v.item(), step_idx)
```

Finally, we calculate the statistics about the gradients on this training step. It's usually good practice to show the graph of the maximum and L2 norm of gradients to get an idea about the training dynamics.

``` python
grad_max = 0.0
grad_means = 0.0
grad_count = 0
for p in net.parameters():
grad_max = max(grad_max, p.grad.abs().max().item())
grad_means += (p.grad ** 2).mean().sqrt().item()
grad_count += 1
```

In [5]:
env = gym.make("CartPole-v0")
writer = SummaryWriter(comment="-cartpole-pg")

net = PGN(env.observation_space.shape[0], env.action_space.n)
print(net)

agent = ptan.agent.PolicyAgent(net, preprocessor=ptan.agent.float32_preprocessor,
                               apply_softmax=True)
exp_source = ptan.experience.ExperienceSourceFirstLast(
    env, agent, gamma=GAMMA, steps_count=REWARD_STEPS)

optimizer = optim.Adam(net.parameters(), lr=LEARNING_RATE)

total_rewards = []
step_rewards = []
step_idx = 0
done_episodes = 0
reward_sum = 0.0
bs_smoothed = entropy = l_entropy = l_policy = l_total = None

batch_states, batch_actions, batch_scales = [], [], []

for step_idx, exp in enumerate(exp_source):
    reward_sum += exp.reward
    baseline = reward_sum / (step_idx + 1)
    writer.add_scalar("baseline", baseline, step_idx)
    batch_states.append(exp.state)
    batch_actions.append(int(exp.action))
    batch_scales.append(exp.reward - baseline)

    # handle new rewards
    new_rewards = exp_source.pop_total_rewards()
    if new_rewards:
        done_episodes += 1
        reward = new_rewards[0]
        total_rewards.append(reward)
        mean_rewards = float(np.mean(total_rewards[-100:]))
        print("%d: reward: %6.2f, mean_100: %6.2f, episodes: %d" % (
            step_idx, reward, mean_rewards, done_episodes))
        writer.add_scalar("reward", reward, step_idx)
        writer.add_scalar("reward_100", mean_rewards, step_idx)
        writer.add_scalar("episodes", done_episodes, step_idx)
        if mean_rewards > 195:
            print("Solved in %d steps and %d episodes!" % (step_idx, done_episodes))
            break

    if len(batch_states) < BATCH_SIZE:
        continue

    states_v = torch.FloatTensor(batch_states)
    batch_actions_t = torch.LongTensor(batch_actions)
    batch_scale_v = torch.FloatTensor(batch_scales)

    optimizer.zero_grad()
    logits_v = net(states_v)
    log_prob_v = F.log_softmax(logits_v, dim=1)
    log_prob_actions_v = batch_scale_v * log_prob_v[range(BATCH_SIZE), batch_actions_t]
    loss_policy_v = -log_prob_actions_v.mean()

    prob_v = F.softmax(logits_v, dim=1)
    entropy_v = -(prob_v * log_prob_v).sum(dim=1).mean()
    entropy_loss_v = -ENTROPY_BETA * entropy_v
    loss_v = loss_policy_v + entropy_loss_v

    loss_v.backward()
    optimizer.step()

    # calc KL-div
    new_logits_v = net(states_v)
    new_prob_v = F.softmax(new_logits_v, dim=1)
    kl_div_v = -((new_prob_v / prob_v).log() * prob_v).sum(dim=1).mean()
    writer.add_scalar("kl", kl_div_v.item(), step_idx)

    grad_max = 0.0
    grad_means = 0.0
    grad_count = 0
    for p in net.parameters():
        grad_max = max(grad_max, p.grad.abs().max().item())
        grad_means += (p.grad ** 2).mean().sqrt().item()
        grad_count += 1

    bs_smoothed = smooth(bs_smoothed, np.mean(batch_scales))
    entropy = smooth(entropy, entropy_v.item())
    l_entropy = smooth(l_entropy, entropy_loss_v.item())
    l_policy = smooth(l_policy, loss_policy_v.item())
    l_total = smooth(l_total, loss_v.item())

    writer.add_scalar("baseline", baseline, step_idx)
    writer.add_scalar("entropy", entropy, step_idx)
    writer.add_scalar("loss_entropy", l_entropy, step_idx)
    writer.add_scalar("loss_policy", l_policy, step_idx)
    writer.add_scalar("loss_total", l_total, step_idx)
    writer.add_scalar("grad_l2", grad_means / grad_count, step_idx)
    writer.add_scalar("grad_max", grad_max, step_idx)
    writer.add_scalar("batch_scales", bs_smoothed, step_idx)

    batch_states.clear()
    batch_actions.clear()
    batch_scales.clear()

writer.close()

PGN(
  (net): Sequential(
    (0): Linear(in_features=4, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=2, bias=True)
  )
)
38: reward:  38.00, mean_100:  38.00, episodes: 1
70: reward:  32.00, mean_100:  35.00, episodes: 2
140: reward:  70.00, mean_100:  46.67, episodes: 3
216: reward:  76.00, mean_100:  54.00, episodes: 4
239: reward:  23.00, mean_100:  47.80, episodes: 5
255: reward:  16.00, mean_100:  42.50, episodes: 6
283: reward:  28.00, mean_100:  40.43, episodes: 7
302: reward:  19.00, mean_100:  37.75, episodes: 8
318: reward:  16.00, mean_100:  35.33, episodes: 9
354: reward:  36.00, mean_100:  35.40, episodes: 10
367: reward:  13.00, mean_100:  33.36, episodes: 11
386: reward:  19.00, mean_100:  32.17, episodes: 12
426: reward:  40.00, mean_100:  32.77, episodes: 13
437: reward:  11.00, mean_100:  31.21, episodes: 14
498: reward:  61.00, mean_100:  33.20, episodes: 15
542: reward:  44.00, mean_100:  33.88, episodes: 16
588: reward

14359: reward: 177.00, mean_100: 126.07, episodes: 152
14559: reward: 200.00, mean_100: 127.67, episodes: 153
14697: reward: 138.00, mean_100: 128.57, episodes: 154
14839: reward: 142.00, mean_100: 129.14, episodes: 155
14967: reward: 128.00, mean_100: 130.11, episodes: 156
15135: reward: 168.00, mean_100: 131.38, episodes: 157
15286: reward: 151.00, mean_100: 132.68, episodes: 158
15416: reward: 130.00, mean_100: 133.59, episodes: 159
15566: reward: 150.00, mean_100: 134.71, episodes: 160
15766: reward: 200.00, mean_100: 135.92, episodes: 161
15966: reward: 200.00, mean_100: 137.39, episodes: 162
16085: reward: 119.00, mean_100: 137.84, episodes: 163
16285: reward: 200.00, mean_100: 139.38, episodes: 164
16485: reward: 200.00, mean_100: 141.10, episodes: 165
16685: reward: 200.00, mean_100: 142.68, episodes: 166
16858: reward: 173.00, mean_100: 144.19, episodes: 167
17034: reward: 176.00, mean_100: 145.13, episodes: 168
17234: reward: 200.00, mean_100: 146.96, episodes: 169
17434: rew

41300: reward: 200.00, mean_100: 183.47, episodes: 301
41500: reward: 200.00, mean_100: 183.47, episodes: 302
41700: reward: 200.00, mean_100: 183.47, episodes: 303
41900: reward: 200.00, mean_100: 183.47, episodes: 304
42100: reward: 200.00, mean_100: 183.47, episodes: 305
42300: reward: 200.00, mean_100: 183.47, episodes: 306
42487: reward: 187.00, mean_100: 183.34, episodes: 307
42687: reward: 200.00, mean_100: 183.34, episodes: 308
42860: reward: 173.00, mean_100: 183.07, episodes: 309
42982: reward: 122.00, mean_100: 182.29, episodes: 310
43089: reward: 107.00, mean_100: 181.36, episodes: 311
43220: reward: 131.00, mean_100: 180.67, episodes: 312
43335: reward: 115.00, mean_100: 179.82, episodes: 313
43448: reward: 113.00, mean_100: 178.95, episodes: 314
43571: reward: 123.00, mean_100: 178.18, episodes: 315
43735: reward: 164.00, mean_100: 177.82, episodes: 316
43879: reward: 144.00, mean_100: 177.26, episodes: 317
44079: reward: 200.00, mean_100: 177.48, episodes: 318
44279: rew

67982: reward: 200.00, mean_100: 176.30, episodes: 451
68182: reward: 200.00, mean_100: 176.30, episodes: 452
68382: reward: 200.00, mean_100: 176.30, episodes: 453
68582: reward: 200.00, mean_100: 176.30, episodes: 454
68782: reward: 200.00, mean_100: 176.30, episodes: 455
68982: reward: 200.00, mean_100: 176.30, episodes: 456
69182: reward: 200.00, mean_100: 176.30, episodes: 457
69382: reward: 200.00, mean_100: 176.30, episodes: 458
69582: reward: 200.00, mean_100: 176.30, episodes: 459
69782: reward: 200.00, mean_100: 176.30, episodes: 460
69982: reward: 200.00, mean_100: 176.30, episodes: 461
70182: reward: 200.00, mean_100: 176.30, episodes: 462
70373: reward: 191.00, mean_100: 176.21, episodes: 463
70499: reward: 126.00, mean_100: 175.47, episodes: 464
70650: reward: 151.00, mean_100: 174.98, episodes: 465
70825: reward: 175.00, mean_100: 174.73, episodes: 466
71025: reward: 200.00, mean_100: 174.73, episodes: 467
71148: reward: 123.00, mean_100: 173.96, episodes: 468
71289: rew

90616: reward: 146.00, mean_100: 150.07, episodes: 601
90777: reward: 161.00, mean_100: 150.33, episodes: 602
90964: reward: 187.00, mean_100: 150.89, episodes: 603
91120: reward: 156.00, mean_100: 151.14, episodes: 604
91303: reward: 183.00, mean_100: 151.62, episodes: 605
91459: reward: 156.00, mean_100: 151.90, episodes: 606
91608: reward: 149.00, mean_100: 151.84, episodes: 607
91760: reward: 152.00, mean_100: 151.92, episodes: 608
91952: reward: 192.00, mean_100: 152.42, episodes: 609
92130: reward: 178.00, mean_100: 153.85, episodes: 610
92328: reward: 198.00, mean_100: 154.53, episodes: 611
92502: reward: 174.00, mean_100: 154.74, episodes: 612
92674: reward: 172.00, mean_100: 155.03, episodes: 613
92819: reward: 145.00, mean_100: 155.32, episodes: 614
92955: reward: 136.00, mean_100: 155.53, episodes: 615
93098: reward: 143.00, mean_100: 155.90, episodes: 616
93216: reward: 118.00, mean_100: 155.91, episodes: 617
93349: reward: 133.00, mean_100: 156.16, episodes: 618
93475: rew

...

131476: reward:  97.00, mean_100: 122.98, episodes: 896
131583: reward: 107.00, mean_100: 122.97, episodes: 897
131693: reward: 110.00, mean_100: 123.19, episodes: 898
131802: reward: 109.00, mean_100: 123.36, episodes: 899
131921: reward: 119.00, mean_100: 123.49, episodes: 900
132047: reward: 126.00, mean_100: 123.73, episodes: 901
132174: reward: 127.00, mean_100: 123.86, episodes: 902
132308: reward: 134.00, mean_100: 124.19, episodes: 903
132443: reward: 135.00, mean_100: 124.33, episodes: 904
132574: reward: 131.00, mean_100: 124.58, episodes: 905
132710: reward: 136.00, mean_100: 124.92, episodes: 906
132853: reward: 143.00, mean_100: 125.37, episodes: 907
133007: reward: 154.00, mean_100: 125.83, episodes: 908
133150: reward: 143.00, mean_100: 126.23, episodes: 909
133298: reward: 148.00, mean_100: 126.73, episodes: 910
133448: reward: 150.00, mean_100: 127.17, episodes: 911
133602: reward: 154.00, mean_100: 127.63, episodes: 912
133733: reward: 131.00, mean_100: 127.84, episod

151662: reward: 166.00, mean_100: 137.06, episodes: 1043
151815: reward: 153.00, mean_100: 137.20, episodes: 1044
151963: reward: 148.00, mean_100: 137.27, episodes: 1045
152119: reward: 156.00, mean_100: 137.28, episodes: 1046
152272: reward: 153.00, mean_100: 137.46, episodes: 1047
152416: reward: 144.00, mean_100: 137.68, episodes: 1048
152575: reward: 159.00, mean_100: 137.87, episodes: 1049
152709: reward: 134.00, mean_100: 137.87, episodes: 1050
152860: reward: 151.00, mean_100: 138.11, episodes: 1051
153006: reward: 146.00, mean_100: 138.27, episodes: 1052
153152: reward: 146.00, mean_100: 138.59, episodes: 1053
153306: reward: 154.00, mean_100: 138.81, episodes: 1054
153455: reward: 149.00, mean_100: 139.12, episodes: 1055
153619: reward: 164.00, mean_100: 139.42, episodes: 1056
153771: reward: 152.00, mean_100: 139.72, episodes: 1057
153930: reward: 159.00, mean_100: 140.14, episodes: 1058
154085: reward: 155.00, mean_100: 140.51, episodes: 1059
154241: reward: 156.00, mean_10

176830: reward: 162.00, mean_100: 188.47, episodes: 1187
176992: reward: 162.00, mean_100: 188.84, episodes: 1188
177157: reward: 165.00, mean_100: 189.18, episodes: 1189
177330: reward: 173.00, mean_100: 189.59, episodes: 1190
177503: reward: 173.00, mean_100: 189.97, episodes: 1191
177688: reward: 185.00, mean_100: 190.33, episodes: 1192
177849: reward: 161.00, mean_100: 190.44, episodes: 1193
178022: reward: 173.00, mean_100: 190.57, episodes: 1194
178210: reward: 188.00, mean_100: 190.57, episodes: 1195
178376: reward: 166.00, mean_100: 190.23, episodes: 1196
178528: reward: 152.00, mean_100: 189.97, episodes: 1197
178698: reward: 170.00, mean_100: 189.93, episodes: 1198
178858: reward: 160.00, mean_100: 189.59, episodes: 1199
179002: reward: 144.00, mean_100: 189.19, episodes: 1200
179161: reward: 159.00, mean_100: 188.78, episodes: 1201
179303: reward: 142.00, mean_100: 188.47, episodes: 1202
179449: reward: 146.00, mean_100: 188.20, episodes: 1203
179591: reward: 142.00, mean_10

197857: reward: 141.00, mean_100: 143.02, episodes: 1332
198024: reward: 167.00, mean_100: 143.11, episodes: 1333
198181: reward: 157.00, mean_100: 142.97, episodes: 1334
198348: reward: 167.00, mean_100: 142.95, episodes: 1335
198504: reward: 156.00, mean_100: 142.90, episodes: 1336
198662: reward: 158.00, mean_100: 142.76, episodes: 1337
198825: reward: 163.00, mean_100: 142.69, episodes: 1338
198985: reward: 160.00, mean_100: 142.64, episodes: 1339
199171: reward: 186.00, mean_100: 143.07, episodes: 1340
199356: reward: 185.00, mean_100: 143.38, episodes: 1341
199515: reward: 159.00, mean_100: 143.58, episodes: 1342
199666: reward: 151.00, mean_100: 143.46, episodes: 1343
199810: reward: 144.00, mean_100: 143.37, episodes: 1344
199991: reward: 181.00, mean_100: 143.70, episodes: 1345
200166: reward: 175.00, mean_100: 143.85, episodes: 1346
200359: reward: 193.00, mean_100: 144.18, episodes: 1347
200559: reward: 200.00, mean_100: 144.34, episodes: 1348
200750: reward: 191.00, mean_10

223166: reward: 126.00, mean_100: 175.62, episodes: 1476
223299: reward: 133.00, mean_100: 175.41, episodes: 1477
223424: reward: 125.00, mean_100: 175.05, episodes: 1478
223545: reward: 121.00, mean_100: 174.68, episodes: 1479
223674: reward: 129.00, mean_100: 174.28, episodes: 1480
223799: reward: 125.00, mean_100: 173.76, episodes: 1481
223917: reward: 118.00, mean_100: 173.00, episodes: 1482
224052: reward: 135.00, mean_100: 172.58, episodes: 1483
224189: reward: 137.00, mean_100: 172.34, episodes: 1484
224325: reward: 136.00, mean_100: 171.77, episodes: 1485
224488: reward: 163.00, mean_100: 171.62, episodes: 1486
224640: reward: 152.00, mean_100: 171.24, episodes: 1487
224771: reward: 131.00, mean_100: 170.55, episodes: 1488
224928: reward: 157.00, mean_100: 170.12, episodes: 1489
225090: reward: 162.00, mean_100: 169.74, episodes: 1490
225270: reward: 180.00, mean_100: 169.55, episodes: 1491
225444: reward: 174.00, mean_100: 169.40, episodes: 1492
225642: reward: 198.00, mean_10